In [1]:
%load_ext autoreload
%autoreload 2
%load_ext autotime

In [45]:
import sys
sys.path.append('../src')
from main import load_data
from preprocessing import clean_text, remove_stop_words, negation_handling, lemmatizing, emoji_tagging
from features_extractor import Extractor, extract_tf, extract_tf_idf, extract_sentiment
from feature_selection import k_best
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score
from util import curry
from pprint import pprint

import matplotlib.pyplot as plt
import copy

time: 4.53 ms


In [19]:
Xtr_text, Ytr, Xte_text, Yte = load_data('../aclImdb/train/', '../aclImdb/test/')

Load Data...
Done loading data!

time: 19.3 s


In [33]:
ngram_range = (1,1)
min_df=2
max_df=0.5
n_features=10000

time: 1.26 ms


In [34]:
Xtr_tfidf, Xte_tfidf = Extractor(Xtr_text, Xte_text)\
        .bind(curry(emoji_tagging))\
        .bind(curry(remove_stop_words))\
        .bind(curry(lemmatizing))\
        .bind(curry(negation_handling))\
        .bind(curry(clean_text))\
        .bind(extract_tf(
                ngram_range=ngram_range,
                min_df=min_df,
                max_df=max_df)
             )\
        .bind(extract_tf_idf)\
        .bind(k_best(Ytr, Yte, n_features))\
        .get_features()

time: 5min 40s


In [39]:
Xtr_tf, Xte_tf = Extractor(Xtr_text, Xte_text)\
        .bind(curry(emoji_tagging))\
        .bind(curry(remove_stop_words))\
        .bind(curry(lemmatizing))\
        .bind(curry(negation_handling))\
        .bind(curry(clean_text))\
        .bind(extract_tf(
                ngram_range=ngram_range,
                min_df=min_df,
                max_df=max_df)
             )\
        .bind(k_best(Ytr, Yte, n_features))\
        .get_features()

time: 5min 53s


In [38]:
Xtr_sent, Xte_sent  = Extractor(Xtr_text, Xte_text)\
        .bind(curry(emoji_tagging))\
        .bind(curry(remove_stop_words))\
        .bind(curry(lemmatizing))\
        .bind(curry(negation_handling))\
        .bind(curry(clean_text))\
        .bind(extract_sentiment)\
        .get_features()

time: 7min 15s


In [41]:
def run_classifier(learner, Xtr, Ytr, Xte, Yte):
    classifier.fit(Xtr, Ytr)
    Yte_hat = classifier.predict(Xte)

    f1 = f1_score(Yte, Yte_hat)
    p = precision_score(Yte, Yte_hat)
    r = recall_score(Yte, Yte_hat)
    return [f1, p, r]

time: 11.1 ms


In [46]:
classifier = LogisticRegression(solver='lbfgs')
score_tf = run_classifier(classifier, Xtr_tf, Ytr, Xte_tf, Yte)
score_tfidf = run_classifier(classifier, Xtr_tfidf, Ytr, Xte_tfidf, Yte)
score_sent = run_classifier(classifier, Xtr_sent, Ytr, Xte_sent, Yte)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


time: 1.69 s


In [47]:
print(score_tf)
print(score_tfidf)
print(score_sent)

[0.8675389710039578, 0.875958244984505, 0.85928]
[0.8856641683941956, 0.8827082008900191, 0.88864]
[0.7188135390503528, 0.7164428196773425, 0.7212]
time: 9.05 ms
